## Imports

In [29]:
import pandas as pd
import numpy as np
import os
import folium
import ast
import textwrap
import googlemaps
from folium.plugins import Fullscreen, MiniMap


## Data Read in

In [17]:
df = pd.read_csv("485x map  - Map_Data_Clean.csv")

In [18]:
df.columns

Index(['NO.', 'FORM SUBMISSION DATE', 'REPORTED PROPERTY ADDRESS',
       'REPORTED PROPERTY BOROUGH', 'REPORTED DOB BIN NUMBER',
       'REPORTED DOB PERMIT SEQUENCE', 'REPORTED PROPERTY TAX BLOCK',
       'REPORTED PROPERTY TAX LOT', 'REPORTED UNITS',
       'REPORTED RESTRICTED UNITS', 'REPORTED COMMENCEMENT DATE',
       'REPORTED ANTICIPATED COMPLETION DATE', 'REPORTED AFFORDABILITY OPTION',
       'PRESUMED COMMUNITY BOARD', 'PRESUMED DUPLICATE', 'DUPLICATE COUNT',
       'PRESUMED BUILDING UNITS', 'PRESUMED BUILDING RESTRICTED UNITS',
       'PRESUMED BBL', 'POSTCODE', 'LATITUDE', 'LONGITUDE', 'COUNCIL DISTRICT',
       'CENSUS TRACT (2020)', 'NEIGHBORHOOD TABULATION AREA (NTA) (2020)'],
      dtype='object')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 25 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   NO.                                        118 non-null    int64  
 1   FORM SUBMISSION DATE                       118 non-null    object 
 2   REPORTED PROPERTY ADDRESS                  118 non-null    object 
 3   REPORTED PROPERTY BOROUGH                  118 non-null    object 
 4   REPORTED DOB BIN NUMBER                    118 non-null    int64  
 5   REPORTED DOB PERMIT SEQUENCE               118 non-null    object 
 6   REPORTED PROPERTY TAX BLOCK                118 non-null    int64  
 7   REPORTED PROPERTY TAX LOT                  118 non-null    int64  
 8   REPORTED UNITS                             118 non-null    int64  
 9   REPORTED RESTRICTED UNITS                  118 non-null    int64  
 10  REPORTED COMMENCEMENT DATE

## Get Coordinates

In [20]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

%store -r map_box_api_key

In [21]:
df['full_Address'] = df['REPORTED PROPERTY ADDRESS'] + ' ' + df['REPORTED PROPERTY BOROUGH'] + ' NY'

In [22]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['full_Address'].apply(geocode)

In [23]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

I think we'd just want the address, presumed unit count & the restricted (affordable) units.

In [25]:
df.columns

Index(['NO.', 'FORM SUBMISSION DATE', 'REPORTED PROPERTY ADDRESS',
       'REPORTED PROPERTY BOROUGH', 'REPORTED DOB BIN NUMBER',
       'REPORTED DOB PERMIT SEQUENCE', 'REPORTED PROPERTY TAX BLOCK',
       'REPORTED PROPERTY TAX LOT', 'REPORTED UNITS',
       'REPORTED RESTRICTED UNITS', 'REPORTED COMMENCEMENT DATE',
       'REPORTED ANTICIPATED COMPLETION DATE', 'REPORTED AFFORDABILITY OPTION',
       'PRESUMED COMMUNITY BOARD', 'PRESUMED DUPLICATE', 'DUPLICATE COUNT',
       'PRESUMED BUILDING UNITS', 'PRESUMED BUILDING RESTRICTED UNITS',
       'PRESUMED BBL', 'POSTCODE', 'LATITUDE', 'LONGITUDE', 'COUNCIL DISTRICT',
       'CENSUS TRACT (2020)', 'NEIGHBORHOOD TABULATION AREA (NTA) (2020)',
       'full_Address', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [26]:
columns_to_keep = [
    'REPORTED PROPERTY ADDRESS', 'REPORTED PROPERTY BOROUGH', 'PRESUMED BUILDING UNITS',
    'PRESUMED BUILDING RESTRICTED UNITS','full_Address', 'geocoded', 'lat',
       'lon'
]

map_df = df[columns_to_keep]

In [27]:
map_df.columns

Index(['REPORTED PROPERTY ADDRESS', 'REPORTED PROPERTY BOROUGH',
       'PRESUMED BUILDING UNITS', 'PRESUMED BUILDING RESTRICTED UNITS',
       'full_Address', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [39]:
df[df['REPORTED PROPERTY ADDRESS'] == '43-09 52 STREET']

,NO.,FORM SUBMISSION DATE,REPORTED PROPERTY ADDRESS,REPORTED PROPERTY BOROUGH,REPORTED DOB BIN NUMBER,REPORTED DOB PERMIT SEQUENCE,REPORTED PROPERTY TAX BLOCK,REPORTED PROPERTY TAX LOT,REPORTED UNITS,REPORTED RESTRICTED UNITS,...,POSTCODE,LATITUDE,LONGITUDE,COUNCIL DISTRICT,CENSUS TRACT (2020),NEIGHBORHOOD TABULATION AREA (NTA) (2020),full_Address,geocoded,lat,lon
24,25,07/25/2024,43-09 52 STREET,QUEENS,4030868,Q00785002,1321,19,33,11,...,11377.0,40.743858,-73.912533,26.0,25302.0,QN0202,43-09 52 STREET QUEENS NY,"(40.743756, -73.912266)",40.743756,-73.912266


In [40]:
# 1) Popup HTML
def create_popup(row):
    lines = [
        f"<h3>{row['REPORTED PROPERTY ADDRESS']}</h3>",
        f"<strong># of Units:</strong> {row['PRESUMED BUILDING UNITS']}<br>",
        f"<strong># of Affordable Units:</strong> {row['PRESUMED BUILDING RESTRICTED UNITS']}<br>"
    ]
    return "<div class='popup-content'>\n" + "\n".join(lines) + "\n</div>"

# 2) Initialize map
first_lat = float(map_df['lat'].iloc[24])
first_lon = float(map_df['lon'].iloc[24])
m = folium.Map(
    location=[first_lat, first_lon],
    zoom_start=10.5,
    scrollWheelZoom=False
)

# 3) Mapbox tile layer
folium.TileLayer(
    tiles=(
        f"https://api.mapbox.com/styles/v1/"
        f"mapbox/streets-v11/tiles/256/{{z}}/{{x}}/{{y}}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox",
    name="Streets",
    overlay=True,
    control=False,
    show=False,
    min_zoom=1,
    max_zoom=20
).add_to(m)

# 4) Optional CSS & Title
custom_css = """
<style>
  .popup-content { min-width:300px; font-size:14px; color:#333; }
  .leaflet-popup-content-wrapper { background:#f9f9f9; border:1px solid #bbb; }
  .leaflet-popup-tip { background:#f9f9f9; }
</style>
"""
m.get_root().html.add_child(folium.Element(custom_css))

title_html = """
<h3 style="text-align:center;
           font-family:Arial,sans-serif;
           font-size:18px;
           color:#333;
           margin-top:10px;">
  <b>Projects seeking 485x</b>
</h3>
"""
m.get_root().html.add_child(folium.Element(title_html))

# 5) Add controls
Fullscreen().add_to(m)
MiniMap(toggle_display=True).add_to(m)

# 6) Fixed radius for all bubbles
fixed_radius = 8  # pixels; tweak as needed

# 7) Add CircleMarkers
for _, row in map_df.iterrows():
    lat, lon = float(row['lat']), float(row['lon'])
    popup = folium.Popup(create_popup(row), max_width=300)

    folium.CircleMarker(
        location=[lat, lon],
        radius=fixed_radius,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        weight=1,
        popup=popup
    ).add_to(m)

# 8) Display
m  # in Jupyter this will render the interactive map

In [35]:
m.save('index.html')

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/NYC_485x
